In [2]:
import os
import utils
import random
import dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import tensorflow.contrib.keras as tf_keras
from keras.models import Model
from keras.layers import Input, Dense

Using TensorFlow backend.


In [3]:
# configst
#tfe.enable_eager_execution()
sess=tf.Session()
%matplotlib inline

# load data
X=dataset.load_dataset("./ShapeNet",num_of_examples=2)
Y=None;

# net parameters
conv_filter_count = [96, 128, 256, 256, 256, 256]
deconv_filter_count = [128, 128, 128, 64, 32, 2]

[load_dataset] loading from ./ShapeNet
[construct_paths] parsing dir ./ShapeNet for ['.obj', '.mtl'] ...
... done
[write_renders_to_disk] writing renders to ./ShapeNet_Renders/02691156/c48af98d47f76002deed0e4a55ad5dd6 ... 
... done
[fetch_renders_from_disk] fetching renders from ./ShapeNet_Renders/02691156/c48af98d47f76002deed0e4a55ad5dd6 ... 
... done
[write_renders_to_disk] writing renders to ./ShapeNet_Renders/02691156/123bd9e948881939c38a1d3458dafa1b ... 
... done
[fetch_renders_from_disk] fetching renders from ./ShapeNet_Renders/02691156/123bd9e948881939c38a1d3458dafa1b ... 
... done


In [4]:
# encoder network
cur_tensor=tf.convert_to_tensor(X[0],dtype=tf.float32)
print(cur_tensor.shape)
k_s = [3,3]
for i in range(6): 
    ks=[7,7]if i is 0 else k_s  
    #with tf.name_scope("encoder_layer"):
    cur_tensor=tf_keras.layers.Conv2D(filters=conv_filter_count[i],kernel_size= k_s)(cur_tensor)
    cur_tensor=tf_keras.layers.MaxPooling2D()(cur_tensor)
    cur_tensor=tf_keras.layers.Activation('relu')(cur_tensor)
    print(cur_tensor.shape)
    
# flatten tensor before 3D-LSTM cells
render_count=cur_tensor.get_shape().as_list()[0]
cur_tensor=tf.reshape(cur_tensor,[render_count,1,-1])
print(cur_tensor.shape)
cur_tensor=tf_keras.layers.Dense(1024)(cur_tensor)
print(cur_tensor.shape)
feature_vector=cur_tensor

(10, 512, 512, 3)
(10, 255, 255, 96)
(10, 126, 126, 128)
(10, 62, 62, 256)
(10, 30, 30, 256)
(10, 14, 14, 256)
(10, 6, 6, 256)
(10, 1, 9216)
(10, 1, 1024)


In [ ]:
# RNN
cur_tensor=tf_keras.layers.GRU(1024)(cur_tensor)
print(cur_tensor.shape)

# decoder network
for i in range(6):
    with tf.name_scope("decoder_layer"):
        cur_tensor=tf_keras.layers.Convolution3DTranspose(filters=deconv_filter_count[i],kernel_size=(3,3,3))(cur_tensor)
        cur_tensor=tf_keras.layers.UpSampling3D()(cur_tensor)
        cur_tensor=tf_keras.layers.Activation('relu')(cur_tensor)
        print(cur_tensor.shape)
    
print(cur_tensor.shape())

In [ ]:
# serialize graph for tensorboard
writer = tf.summary.FileWriter("./logs/")
writer.add_graph(sess.graph)